# Projeto 1 - Ciência dos Dados

Nome: Paulo Kim

Nome: Rodrigo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [26]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import math

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [27]:
import os

filename = 'iPhone_X.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPhone_X.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados de treinamento com os tweets classificados:

In [47]:
train = pd.read_excel(filename)
train = train.iloc[:,:2]
train.head(5)

,Treinamento,Classificação
0,@iconredesign my prediction is the same but:\n...,2
1,still have mine but i never connected to my ph...,1
2,"iphone 11 na waste of money, just chill with x...",2
3,once the iphone 12 drops those using the iphon...,2
4,#ncat who fix’s phone ?? it’s a iphone x,1


Carregando a base de dados de teste com os tweets classificados:

In [48]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test = test.iloc[:,:2]
test.head()

,Teste,Classificação
0,@starrynight024 right?? that’s exactly what ha...,3
1,@gossipgirliee some of us haven't bought iphone x,2
2,@manyaww @mlgcollegefess hs without cond*m and...,1
3,very annoying that they didn't announce the ne...,2
4,@clarkvolo iphone x was released in november tho,2


___
## Classificador automático de sentimento


Nós pegamos como produto o iPhone X. Para sua classificação, dividimos os tweets em:

* **Propaganda/oferta** (0) --> tweets com anúncios de valor e venda de iPhones, assim como vendas de peliculas e capas específicas para o iPhone X
* **Muito irrelevante** (1) --> tweets que mencionavam o iPhone X, mas que nada relacionado ao aparelho em si, apenas citando ele
* **Irrelevante** (2) --> tweets que além de mencionar o iPhone, faziam comentários relacionados ao aparelho, mas muito superficiais
* **Relevante** (3) --> tweets que traziam alguma carga de crítica ao aparelho (seja boa ou ruim)
* **Muito relevante** (4) --> tweets com altas cargas criticas construtivas, que especificam algum parte ou recurso do aparelho

In [49]:
# Função cleanup que realiza a filtragem de caracteres desnecessários, 
# assim como menção à contas e referências a sites

def cleanup(text):
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    text_subbed_list = text_subbed.split()
    
    for word in text_subbed_list:
        if word[0] == '@' or word[:5] == 'https':
            text_subbed_list.remove(word)
        else:
            pass
    return ' '.join(text_subbed_list)  

In [55]:
# Loop para realizar a limpeza do treinamento
for i in range(len(train)):
    train.at[i,'Treinamento'] = cleanup(train.Treinamento[i]).lower() 
                                                                      
train.head()

,Treinamento,Classificação
0,my prediction is the same but a13 bionic face ...,2
1,still have mine but i never connected to my ph...,1
2,iphone 11 na waste of money just chill with x ...,2
3,once the iphone 12 drops those using the iphon...,2
4,ncat who fix’s phone it’s a iphone x,1


In [56]:
# Loop para realizar a limpeza do teste
for i in range(len(test)):
    test.at[i,'Teste'] = cleanup(test.Teste[i]).lower() 
                                                                      
test.head()

,Teste,Classificação
0,right that’s exactly what happened when i got ...,3
1,some of us havent bought iphone x,2
2,hs without condm and what did you expected to ...,1
3,very annoying that they didnt announce the new...,2
4,iphone x was released in november tho,2


In [136]:
# Calculando o total de palavras no conjunto dos tweets da base de treinamento
total_palavras = 0 
for phrase in train.Treinamento:
    total_palavras += len(phrase.split())

___
### Montagens das séries e tabelas relativas para cada classificação

Filtragem do dataframe para cada classficação, para assim montar as tabelas com as probabilidades de cada palavra dada uma classificação


In [137]:
# Transformando a lista de palavras da classficação Propaganda/Oferta (0) em uma série
lista_palavras_class_0 = []
palavras_class_0 = train.loc[train.Classificação == 0,:]

for phrase in palavras_class_0.Treinamento:
    lista_palavras_class_0 += phrase.split()
    
series_palavras_class_0 = pd.Series(lista_palavras_class_0)

In [138]:
# Transformando a lista de palavras da classficação Muito Irrelevante (1) em uma série
lista_palavras_class_1 = []
palavras_class_1 = train.loc[train.Classificação == 1,:]

for phrase in palavras_class_1.Treinamento:
    lista_palavras_class_1 += phrase.split()
    
series_palavras_class_1 = pd.Series(lista_palavras_class_1)

In [139]:
# Transformando a lista de palavras da classficação Irrelevante (2) em uma série
lista_palavras_class_2 = []
palavras_class_2 = train.loc[train.Classificação == 2,:]

for phrase in palavras_class_2.Treinamento:
    lista_palavras_class_2 += phrase.split()
    
series_palavras_class_2 = pd.Series(lista_palavras_class_2)

In [140]:
# Transformando a lista de palavras da classficação Relevante (3) em uma série
lista_palavras_class_3 = []
palavras_class_3 = train.loc[train.Classificação == 3,:]

for phrase in palavras_class_3.Treinamento:
    lista_palavras_class_3 += phrase.split()
    
series_palavras_class_3 = pd.Series(lista_palavras_class_3)

In [141]:
# Transformando a lista de palavras da classficação Muito Relevante (4) em uma série
lista_palavras_class_4 = []
palavras_class_4 = train.loc[train.Classificação == 4,:]

for phrase in palavras_class_4.Treinamento:
    lista_palavras_class_4 += phrase.split()
    
series_palavras_class_4 = pd.Series(lista_palavras_class_4)

In [142]:
# Calculando a quantidade de cada palavra dada a classificação
qntd_absoluta_palavra_class_0 = series_palavras_class_0.value_counts()
qntd_absoluta_palavra_class_1 = series_palavras_class_1.value_counts()
qntd_absoluta_palavra_class_2 = series_palavras_class_2.value_counts()
qntd_absoluta_palavra_class_3 = series_palavras_class_3.value_counts()
qntd_absoluta_palavra_class_4 = series_palavras_class_4.value_counts()

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [143]:
# Calculando a probabilidade de uma palavra pertencer à respectiva classificação
P_0 = len(train.loc[train['Classificação']== 0,:]) / len(train)
P_1 = len(train.loc[train['Classificação']== 1,:]) / len(train)
P_2 = len(train.loc[train['Classificação']== 2,:]) / len(train)
P_3 = len(train.loc[train['Classificação']== 3,:]) / len(train)
P_4 = len(train.loc[train['Classificação']== 4,:]) / len(train)

In [146]:
# Smoothing
smoothing = {
    'V': 10**4,
    'alpha': 0.1
}

# Funções resposáveis em calcular as probabilidades com a aplicação do smoothing

def prob_frase_dado_class_0(frase):
    prob = 1
    for palavra in frase.split():
        try: # Verifica se a palavra está contida na série da respectiva categoria. 
            prob *= (qntd_absoluta_palavra_class_0[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_0.sum() + smoothing['alpha']*smoothing['V']) 
        except: # Caso a palavra não exista, a técnica de smoothing impede que a probabilidade resulte em zero
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_0.sum() + smoothing['alpha']*smoothing['V'])
        
    return prob

def prob_frase_dado_class_1(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_1[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_1.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_1.sum() + smoothing['alpha']*smoothing['V'])
                                                                                 
    return prob

def prob_frase_dado_class_2(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_2[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_2.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(qntd_absoluta_palavra_class_2) + smoothing['alpha']*smoothing['V'])
                                                                                
    return prob

def prob_frase_dado_class_3(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_3[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_3.sum() + smoothing['alpha']*smoothing['V'])  
        except:
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_3.sum() + smoothing['alpha']*smoothing['V'])
                                                                                
    return prob

def prob_frase_dado_class_4(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_4[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_4.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_4.sum() + smoothing['alpha']*smoothing['V'])
                                                                               
    return prob

In [147]:
def naive_bayes(tweet):
    p_0_dado_frase = math.log(prob_frase_dado_class_0(tweet) * P_0)
    p_1_dado_frase = math.log(prob_frase_dado_class_1(tweet) * P_1)
    p_2_dado_frase = math.log(prob_frase_dado_class_2(tweet) * P_2)
    p_3_dado_frase = math.log(prob_frase_dado_class_3(tweet) * P_3)
    p_4_dado_frase = math.log(prob_frase_dado_class_4(tweet) * P_4)
    
    # As probabilidade são guardadas em uma lista, para assim, retornar o maior valor dela
    lista_de_probabilidades = [p_0_dado_frase,
                               p_1_dado_frase,
                               p_2_dado_frase,
                               p_3_dado_frase,
                               p_4_dado_frase]
    
    #Já que a lista está ordenada de acordo com as categorias, o index da maior probabilidade será a própria classificação
    return lista_de_probabilidades.index(max(lista_de_probabilidades))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [149]:
# Aplicando a função Naive Bayes ao DataFrame de Teste

test.loc[:,'NBayes'] = 0
for i in range(len(test)):
    test.iloc[i,2] = naive_bayes(test.Teste[i])
test

,Teste,Classificação,NBayes
0,right that’s exactly what happened when i got ...,3,2
1,some of us havent bought iphone x,2,1
2,hs without condm and what did you expected to ...,1,2
3,very annoying that they didnt announce the new...,2,3
4,iphone x was released in november tho,2,1
...,...,...,...
720,mojosmartphone qi leather wireless charger,0,0
721,shop anythingyoucanimagine,0,0
722,glowing apple logo backlit led case for iphone...,0,0
723,shit crazy had this iphone x forever,2,3


In [152]:
# Tabela que compara classificação humana e a classificação do algoritmo no Teste

tabela_acuracia_teste = pd.crosstab(test.Classificação, test.NBayes, normalize = True)
tabela_acuracia_teste

NBayes,0,1,2,3,4
Classificação,,,,,
0,0.180690,0.012414,0.023448,0.002759,0.000000
1,0.008276,0.023448,0.064828,0.001379,0.000000
2,0.026207,0.155862,0.228966,0.053793,0.009655
3,0.000000,0.026207,0.089655,0.041379,0.002759
4,0.000000,0.002759,0.026207,0.013793,0.005517


In [153]:
taxa_acuracia_teste = tabela_acuracia_teste.iloc[0,0] + tabela_acuracia_teste.iloc[1,1] + tabela_acuracia_teste.iloc[2,2] + tabela_acuracia_teste.iloc[3,3] 

print('A taxa de acurácia do classificador no teste é de {}%'.format(taxa_acuracia_teste.round(4)*100))

A taxa de acurácia do classificador no teste é de 47.449999999999996%


In [154]:
# Aqui aplicamos a função Naive Bayes ao nosso DataFrame de Treinamento

train.loc[:,'NBayes'] = 0
for i in range(len(train)):
    train.iloc[i,2] = naive_bayes(train.Treinamento[i])

In [155]:
# Tabela que compara classificação humana e a classificação do algoritmo no Treinamento

tabela_acuracia_treino = pd.crosstab(train.Classificação, train.NBayes, normalize = True)
tabela_acuracia_treino

NBayes,0,1,2,3,4
Classificação,,,,,
0,0.2144,0.0040,0.0032,0.0008,0.0000
1,0.0056,0.3384,0.0088,0.0032,0.0008
2,0.0016,0.0240,0.1704,0.0008,0.0000
3,0.0008,0.0032,0.0072,0.1480,0.0000
4,0.0000,0.0016,0.0008,0.0040,0.0584


In [156]:
taxa_acuracia_treino = tabela_acuracia_treino.iloc[0,0] + tabela_acuracia_treino.iloc[1,1] + tabela_acuracia_treino.iloc[2,2] + tabela_acuracia_treino.iloc[3,3] 
print('A taxa de acurácia do classificador no teste é de {}%'.format(taxa_acuracia_treino.round(4)*100))

A taxa de acurácia do classificador no teste é de 87.12%


___
## Concluindo:

Nosso avaliador obteve uma taxa de acerto de 47,5% quando aplicado a uma base de dados Teste (simulando tweets aleatórios pegos no Twitter) e uma taxa de acerto de 87,12% quando aplicado de volta em cima da própria base de dados (Treinamento) que ajudou a treina-lo. Portanto, aproximandando os valores, pegando um tweet aleatório que falasse do iPhoneX, teríamos quase 50% de chance de acertar utilizando nosso classificador.

Algumas ressalvas são para o fato de termos usados muitas categorias e para o fato da própria classificação subjetiva feita por cada um do grupo. As vezes por erros de interpretação tweets podem ter sidos classificados errados, e isso impacta diretamente no algoritmo.

Outro ponto importante é a relação de números absolutos dos tweets da nossa base de treinamento:

* Deal (0): 278 tweets
* Muito irrelevante (1): 446 tweets
* Irrelevante (2): 249 tweets
* Relevante (3): 199 tweets
* Muito relevante (4): 81 tweets

* Total: 1250 tweets avaliados

Com isso, percebemos uma problemática no nosso algoritmo, pois a base de treinamento possui muito mais dados para classificar um tweet nas categorias 0, 1 e 2, mas pouquissimos recursos para classificar o tweet nas categorias 3 e 4. Desse modo, por termos uma base de dados fraca no caso das classificações 3 e 4, sempre que chegava uma palavra que não estava nessa base de dados ela passava por smoothing, enquanto nas classificações 0, 1 e 2 haviam muito mais recursos  para definir a probabilidade da palavra em análise.

De acordo com a base de dados, percebemos uma limitação do algoritmo dada a pouca quantidade de dados fornecida para ele treinar. Como possíveis melhorias do noss modelo, buscaríamos, além de nivelar a quantidade de tweets de cada classificação, aumentar a base de dados, pois assim o algortimo estaria mais preparado para palavras novas, e nós não dependeríamos tanto assim dos processor de smoothing.

O algoritmo de Naive Bayes se mostra como um poderoso classificador, independente do cenário. Ele pode ser usado para classificar sentimento, filtrar spam, sistemas de recomendação. Esse é um algoritmo que depende fortemente da construção correta de sua base de dados. Assim como dito anteriormente, se bem construida pode fornecer um grande potencial para o uso desse algoritmo.

Por fim, como um resumo de tudo falado. poderíamos melhorar o uso do nosso algoritmo incrementando nossa base de dados de modo a nivelar as quantidades de tweets e aumentar, também, a quantidade de tweets pegos. Além disso, podeíamos fazer algumas melhorias na estrutura do código que, em alguns momentos, ficou repetitivo e potencialmente lento. Poderíamos trabalhar com loops e dicionários para resolver a questão do código e trabalhar com mais dados para resolver a problemática da base de dados.

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**